A pipline to transform raw read data in FASTQ format to clean contact data.
Requires FastUniq, Trimmomatic, Hisat2, SAMTools, BedTools.

Defining paths and variables

In [24]:
FastUniqPath=""
TrimmomaticPath=""
Hisat2Path=""
SamtoolsPath=""
BedtoolsPath=""
SRR1=""
SRR2=""
GenomePath=""
GenomeName=""
SpliceSitePath=""
ContactsOutput="1"
CIGARIntersectionOutput=""
Hisat2ThreadNumber=
export PATH=$PATH:$BedtoolsPath

Checking file existences

In [4]:
if [ ! -x "$FastUniqPath" ]; then echo "Wrong FastUniq path"; exit 1; fi
if [ ! -x "$TrimmomaticPath" ]; then echo "Wrong Trimmomatic path"; exit 1; fi
if [ ! -d "$Hisat2Path" ]; then echo "Wrong Hisat2 path"; exit 1; fi
if [ ! -x "$SamtoolsPath" ]; then echo "Wrong Samtools path"; exit 1; fi
if [ ! -d "$BedtoolsPath" ]; then echo "Wrong Bedtools path"; exit 1; fi
if [ ! -e "${SRR1}".fastq ] || [ ! -e "${SRR2}".fastq ]; then echo "Wrong data path"; exit 1; fi
if [ ! -e "$GenomePath" ]; then echo "Wrong genome path"; exit 1; fi

Wrong FastUniq path
exit
Restarting Bash


Using the Arabidopsis genome (TAIR10.1) we index it for further mapping

In [3]:
curl -L https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/735/GCF_000001735.4_TAIR10.1/GCF_000001735.4_TAIR10.1_genomic.fna.gz -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35.7M  100 35.7M    0     0  1765k      0  0:00:20  0:00:20 --:--:-- 2304k


In [5]:
gzip -dk GCF_000001735.4_TAIR10.1_genomic.fna.gz

In [30]:
${Hisat2Path}hisat2-build -p$Hisat2ThreadNumber $GenomePath $GenomeName > /dev/null

Settings:
  Output files: "TAIR10.1.*.ht2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Local offset rate: 3 (one in 8)
  Local fTable chars: 6
  Local sequence length: 57344
  Local sequence overlap between two consecutive indexes: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /home/vanya/slowdisk/arabidopsis/temp/GRID-seq-main/GRID-seq/Identification/data/Final_raw_data/GCF_000001735.4_TAIR10.1_genomic.fna
Reading reference sizes
  Time reading reference sizes: 00:00:02
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
  Time to read SNPs and splice sites: 00:00:00
Using parameters --bmax 746765 --dcv 1024
  Doing ahead-of-time memory usage t

Downloading GTF file to determine splicing sites

In [16]:
curl -L ftp://ftp.ensemblgenomes.org/pub/release-37/plants/gtf/arabidopsis_thaliana/Arabidopsis_thaliana.TAIR10.37.gtf.gz -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9842k  100 9842k    0     0  1971k      0  0:00:04  0:00:04 --:--:-- 2298k


In [18]:
gzip -dk Arabidopsis_thaliana.TAIR10.37.gtf.gz

Creating splicing sites markdown

In [19]:
python3 ${Hisat2Path}hisat2_extract_splice_sites.py Arabidopsis_thaliana.TAIR10.37.gtf > splicesites.txt

Mapping to the reference genome (TAIR10.1) using Hisat2 (DNA part)

In [196]:
${Hisat2Path}hisat2 -f -p$Hisat2ThreadNumber -x$GenomeName -k 100 \
--no-spliced-alignment --no-softclip \
-U ${SRR1}.fa | $SamtoolsPath view -bSh > ${SRR1}.bam 

20689053 reads; of these:
  20689053 (100.00%) were unpaired; of these:
    8579978 (41.47%) aligned 0 times
    9996324 (48.32%) aligned exactly 1 time
    2112751 (10.21%) aligned >1 times
58.53% overall alignment rate


Mapping to the reference genome (Sus11.1) using Hisat2 (RNA part)

In [197]:
${Hisat2Path}hisat2 -f -p$Hisat2ThreadNumber -x$GenomeName -k 100 --no-softclip --dta-cufflinks \
--known-splicesite-infile $SpliceSitePath \
--novel-splicesite-outfile ${SRR2}_novel_splice -U ${SRR2}.fa | $SamtoolsPath view -bSh > ${SRR2}.bam

20689053 reads; of these:
  20689053 (100.00%) were unpaired; of these:
    3335621 (16.12%) aligned 0 times
    4427202 (21.40%) aligned exactly 1 time
    12926230 (62.48%) aligned >1 times
83.88% overall alignment rate


Getting number of reads mapped

In [9]:
$SamtoolsPath view -c -F 260 ${SRR1}.bam

10337216


In [10]:
$SamtoolsPath view -c -F 260 ${SRR2}.bam

16233380


Applying filter to reads. Only unique mappings with at most 2 mismatches remain

In [200]:
$SamtoolsPath view -Sh -F 4 ${SRR1}.bam | grep -E 'XM:i:[0-2]\s.*NH:i:1$|^@' | \
    $SamtoolsPath view -Sbh > ${SRR1}_filtered.bam

In [201]:
$SamtoolsPath view -Sh -F 4 ${SRR2}.bam | grep -E 'XM:i:[0-2]\s.*NH:i:1$|^@' | \
    $SamtoolsPath view -Sbh > ${SRR2}_filtered.bam

Counting filtered reads

In [11]:
$SamtoolsPath view -c -F 260 ${SRR1}_filtered.bam

8315552


In [12]:
$SamtoolsPath view -c -F 260 ${SRR2}_filtered.bam

3111955


Converting to BED format

In [204]:
$SamtoolsPath view -Sh -F 4 ${SRR1}_filtered.bam | $SamtoolsPath view -Sbh | $BedtoolsPath/bamToBed -cigar -i > ${SRR1}.bed

In [205]:
$SamtoolsPath view -Sh -F 4 ${SRR2}_filtered.bam | $SamtoolsPath view -Sbh | $BedtoolsPath/bamToBed -cigar -i > ${SRR2}.bed

Intersecting BED files and forming contacts

In [207]:
cat cnt_intersect.py

import pandas as pd
from sys import argv

dna = pd.read_csv(argv[1], sep = "\t", header = None)
rna = pd.read_csv(argv[2], sep = "\t", header = None)
dna.set_index([3], inplace=True)
rna.set_index([3], inplace=True)
ids = list(set(dna.index) & set(rna.index))                  
dna = dna.loc[ids,]
rna = rna.loc[ids,]
res = pd.DataFrame(index=range(len(ids)),\
columns=['id', 'rna_chr', 'rna_bgn', 'rna_end', 'rna_strand', 'rna_cigar',\
'dna_chr', 'dna_bgn', 'dna_end', 'dna_strand', 'dna_cigar'])
for i, item in enumerate(ids):
    res.at[i,'id'] = item
    res.at[i,'rna_chr'] = rna.at[item,0]
    res.at[i,'rna_bgn'] = rna.at[item,1]
    res.at[i,'rna_end'] = rna.at[item,2]
    res.at[i,'rna_strand'] = rna.at[item,5]
    res.at[i,'rna_cigar'] = rna.at[item,6]
    res.at[i,'dna_chr'] = dna.at[item,0]
    res.at[i,'dna_bgn'] = dna.at[item,1]
    res.at[i,'dna_end'] = dna.at[item,2]
    res.at[i,'dna_strand'] = dna.at[item,5]
    res.at[i,'dna_cigar'] = dna.at[item,6]
res.to_csv(argv[3], sep='

In [208]:
python3 cnt_intersect.py ${SRR1}.bed ${SRR2}.bed $ContactsOutput

Counting contacts

In [25]:
wc -l $ContactsOutput

1709099 1rep_cnt.tsv


CIGAR filtering: reads with a complete match ('digitsM' flag) are left intact, reads with only one mismatch ('digitsMdigitsNdigitsM' type) are cut to their longest match and other reads are dropped

In [213]:
cat cigar_filter.py

import re
from sys import argv


f = open(argv[1], 'r')
f1 = open(argv[2], 'w')
linecnt = 0

c1 = re.compile('\d+M')
c2 = re.compile('(\d+)M(\d+)N(\d+)M')
cnt1 = 0
cnt2 = 0

while True:
    linecnt += 1
    line = f.readline().strip()
    if not line:
        break
    llist = line.split()
    tline = llist[6]
    
    if len(c1.findall(tline)) == 1:
        cnt1 += 1
        print(line, file=f1)
    elif len(c1.findall(tline)) == 2:        
        l2 = c2.findall(tline)
        if len(l2)>0:
            cnt2 += 1
            if int(l2[0][0]) > int(l2[0][2]):
                tmp = int(llist[1]) + int(l2[0][0])
                llist[2] = str(tmp)
                print('\t'.join(llist), file=f1)
            else:
                tmp = int(llist[2]) - int(l2[0][2])
                llist[1] = str(tmp)
                print('\t'.join(llist), file=f1)
f.close()
print(f'full match {cnt1}')
print(f'central gap {cnt2}')



In [214]:
python3 cigar_filter.py ${SRR2}.bed ${SRR2}_filtered.bed

full match 4417114
central gap 10071


In [218]:
python3 cnt_intersect.py ${SRR1}.bed ${SRR2}_filtered.bed $CIGARIntersectionOutput

Counting contacts

In [15]:
wc -l $CIGARIntersectionOutput

1709083 1rep_cigar_out.tsv


Merging replicas into one

In [223]:
cat *cigar*tsv| grep -v rna_bgn > merged_before_rna1.tsv

In [16]:
wc -l merged_before_rna1.tsv

9300643 merged_before_rna1.tsv


Changing chromosome names from RefSeq IDs to usual

In [228]:
sed -i -e 's/NC_003070.9/chr1/g' -e 's/NC_003071.7/chr2/g' -e 's/NC_003074.8/chr3/g' -e 's/NC_003075.7/chr4/g' -e 's/NC_003076.8/chr5/g' -e 's/NC_037304.1/MT/g' -e 's/NC_000932.1/Pltd/g' merged_before_rna1.tsv

A script to transform GTF file into genes markdown

In [156]:
cat genes_tidy.py

from sys import argv


f = open(argv[1], 'r')
f1 = open(argv[2], 'w+')
linecnt = 0

while True:
    linecnt += 1  
    line = f.readline()
    if not line:
        break
    llist = line.split()
    
    try:
        if llist[2] == 'gene':
            print(f'chr{llist[0]}\t{llist[3]}\t{llist[4]}\t{llist[6]}\t{llist[15][1:-2]}\t{llist[11][1:-2]}\t{int(llist[4])-int(llist[3])}', file=f1)
    except:
        continue
f.close()
f1.close()


Transforming:

In [157]:
python3 genes_tidy.py Arabidopsis_thaliana.TAIR10.37.gtf genes1.tsv

RNA type distribution among the genes:

In [161]:
cat genes1.tsv | awk '{print $5}' | sort | uniq -c

      1 antisense_RNA
      1 araport11
     24 ene_bioty
    176 miRNA
     15 nontranslating_CDS
      9 otherRNA
     39 pre_miRNA
  12861 protein_coding
      2 RNase_MRP_RNA
    526 rRNA
     27 sense_intronic
    146 snoRNA
     84 snRNA
     10 SRP_RNA
    403 tRNA


A script to annotate RNA-parts of contacts

In [17]:
cat annotation.sh

# Creating temporary files
t1=$(mktemp)
t2=$(mktemp)

# Contacts and genes paths
ContactsFile='merged_before_rna1.tsv'
GenesFile='genes1.tsv'

# Intersection of contacts and genes, strands are stored in corresponding files; didn't find bedtools on HPC-2 so provided thast files
bedtools intersect -a <(cat $ContactsFile| awk '{print $2, $3, $4}'  OFS='\t') -b <(cat $GenesFile) -wb > intersected_genes.tsv
bedtools intersect -a <(cat $ContactsFile| awk '{print $2, $3, $4, $5, $1}'  OFS='\t') -b <(cat $GenesFile) -wa > intersected_contacts.tsv

# Counting how many contacts intersected certain gene = gene score
cat intersected_genes.tsv | awk '{print $9}' | sort | uniq -c > gene_score.csv

# Adding score column to the intersection from the genes perspective
join -1 9 -2 2 -o1.1,1.2,1.3,1.7,1.8,1.9,1.10,2.1 <(sort -k 9,9 intersected_genes.tsv) gene_score.csv > intersected_scored_genes.csv

# Dividing by gene length
cat intersected_scored_genes.csv | awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$8/$7}'

In [18]:
./annotation.sh

***** WARNING: File /dev/fd/63 has inconsistent naming convention for record:
MT	143842	143859

***** WARNING: File /dev/fd/63 has inconsistent naming convention for record:
MT	143842	143859

***** WARNING: File /dev/fd/63 has inconsistent naming convention for record:
MT	143842	143859	-	8388644

***** WARNING: File /dev/fd/63 has inconsistent naming convention for record:
MT	143842	143859	-	8388644



Counting contacts

In [19]:
wc -l annotated_contacts.csv

5161255 annotated_contacts.csv


RNA distribution after RNA-annotation

In [20]:
cat annotated_contacts.csv | sort -k12,12 |  awk '{print $11,$12}' | uniq -f1 | awk '{print $1}' | sort | uniq -c

      1 araport11
     17 ene_bioty
     73 miRNA
     13 nontranslating_CDS
      7 otherRNA
      7 pre_miRNA
  11494 protein_coding
      1 RNase_MRP_RNA
      2 rRNA
     56 snoRNA
     16 snRNA
      1 SRP_RNA
      9 tRNA


Getting RNAs with the most contacts

In [21]:
cat annotated_contacts.csv | sort -k12,12 |  awk '{print $11,$12}' | uniq -c  > rna_cnt_number1.csv

In [22]:
cat rna_cnt_number1.csv | sort -k1nr,1n | head -n10 

 171868 rRNA SSU_rRNA_eukarya
 107699 snRNA U2
  80503 snRNA U2.4
  27585 snRNA U2.7
  14212 protein_coding RCA
  13516 protein_coding BXL1
  13387 protein_coding TSS
  13076 protein_coding LHCA4
  11481 SRP_RNA Plant_SRP
  11304 protein_coding ABCG36
sort: write failed: 'standard output': Broken pipe
sort: write error


Contacts distribtion after RNA-annotation

In [23]:
cat annotated_contacts.csv  | awk '{print $11}' | sort | uniq -c

    590 araport11
   9997 ene_bioty
   3603 miRNA
   2834 nontranslating_CDS
   7743 otherRNA
    328 pre_miRNA
4660208 protein_coding
    690 RNase_MRP_RNA
 172026 rRNA
  44589 snoRNA
 247028 snRNA
  11481 SRP_RNA
    138 tRNA


Atempting to prepare data to normalization

In [290]:
head npcrna.csv

chr6 71971138 71971157 + 19M chr13 124590001 124590020 - 19M snRNA U5 19686 100002016
chr14 40286509 40286528 - 19M chr6 100028477 100028496 + 19M snRNA U4 8808 100004528
chr1 236380016 236380035 - 19M chr18 11678219 11678239 + 20M ribozyme RNase_MRP 1300 100031442
chr14 40286508 40286527 - 19M chr11 13656391 13656410 - 19M snRNA U4 8808 100036647
chr1 163335027 163335046 + 19M chr14 48515712 48515731 - 19M snRNA U5 19686 100043587
chr14 40286550 40286569 - 19M chr13 199193288 199193307 - 19M snRNA U4 8808 100059528
chr14 40286550 40286569 - 19M chr6 54532506 54532526 + 20M snRNA U4 8808 100070833
chr14 40286550 40286569 - 19M chr6 54532506 54532526 + 20M snRNA U4 8808 100070833
chr6 54567220 54567239 + 19M chr18 16638200 16638219 + 19M snoRNA SNORD33 1228 100084671
chr6 71971138 71971157 + 19M chr1 118912052 118912071 + 19M snRNA U5 19686 100087314


In [310]:
cat pcrna.csv | awk '{print $12}' | sort | uniq -c | sort -k1nr,1 | tail -n+51 | head -n -1000 > fpcrna

In [311]:
wc -l fpcrna

13911 fpcrna


In [314]:
join -1 12 -2 2 <(cat contacts_rna_annotated1.csv | sort -k12,12) <(cat fpcrna | sort -k2,2) > fpcrna_contacts.csv

In [315]:
head fpcrna_contacts.csv

A1CF chr14 98909017 98909036 + 19M chr14 98908799 98908818 - 19M protein_coding 54 245788663 54
A1CF chr14 98910836 98910855 - 19M chr14 98910543 98910562 - 19M protein_coding 54 146253771 54
A1CF chr14 98911368 98911387 + 19M chr16 73406426 73406445 + 19M protein_coding 54 232813212 54
A1CF chr14 98911368 98911387 + 19M chr16 73406426 73406445 + 19M protein_coding 54 232813212 54
A1CF chr14 98913970 98913990 + 20M chr14 98914095 98914115 + 20M protein_coding 54 145406469 54
A1CF chr14 98913970 98913990 + 20M chr14 98914095 98914115 + 20M protein_coding 54 145406469 54
A1CF chr14 98919267 98919286 - 19M chr14 49314303 49314323 + 20M protein_coding 54 95062353 54
A1CF chr14 98919352 98919371 - 19M chr14 99673719 99673738 + 19M protein_coding 54 161146852 54
A1CF chr14 98920675 98920694 - 19M chr14 98920575 98920595 - 20M protein_coding 54 144115271 54
A1CF chr14 98929193 98929212 + 19M chr14 98929419 98929439 + 20M protein_coding 54 8849515 54


In [321]:
cat fpcrna_contacts.csv | awk '{print $2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$1,$13,$14,$15}' OFS='\t' > fpcrna_contacts1.csv

In [322]:
head fpcrna_contacts1.csv

chr14	98909017	98909036	+	19M	chr14	98908799	98908818	-	19M	protein_coding	A1CF	54	245788663	54
chr14	98910836	98910855	-	19M	chr14	98910543	98910562	-	19M	protein_coding	A1CF	54	146253771	54
chr14	98911368	98911387	+	19M	chr16	73406426	73406445	+	19M	protein_coding	A1CF	54	232813212	54
chr14	98911368	98911387	+	19M	chr16	73406426	73406445	+	19M	protein_coding	A1CF	54	232813212	54
chr14	98913970	98913990	+	20M	chr14	98914095	98914115	+	20M	protein_coding	A1CF	54	145406469	54
chr14	98913970	98913990	+	20M	chr14	98914095	98914115	+	20M	protein_coding	A1CF	54	145406469	54
chr14	98919267	98919286	-	19M	chr14	49314303	49314323	+	20M	protein_coding	A1CF	54	95062353	54
chr14	98919352	98919371	-	19M	chr14	99673719	99673738	+	19M	protein_coding	A1CF	54	161146852	54
chr14	98920675	98920694	-	19M	chr14	98920575	98920595	-	20M	protein_coding	A1CF	54	144115271	54
chr14	98929193	98929212	+	19M	chr14	98929419	98929439	+	20M	protein_coding	A1CF	54	8849515	54


In [318]:
head npcrna.csv

chr6 71971138 71971157 + 19M chr13 124590001 124590020 - 19M snRNA U5 19686 100002016
chr14 40286509 40286528 - 19M chr6 100028477 100028496 + 19M snRNA U4 8808 100004528
chr1 236380016 236380035 - 19M chr18 11678219 11678239 + 20M ribozyme RNase_MRP 1300 100031442
chr14 40286508 40286527 - 19M chr11 13656391 13656410 - 19M snRNA U4 8808 100036647
chr1 163335027 163335046 + 19M chr14 48515712 48515731 - 19M snRNA U5 19686 100043587
chr14 40286550 40286569 - 19M chr13 199193288 199193307 - 19M snRNA U4 8808 100059528
chr14 40286550 40286569 - 19M chr6 54532506 54532526 + 20M snRNA U4 8808 100070833
chr14 40286550 40286569 - 19M chr6 54532506 54532526 + 20M snRNA U4 8808 100070833
chr6 54567220 54567239 + 19M chr18 16638200 16638219 + 19M snoRNA SNORD33 1228 100084671
chr6 71971138 71971157 + 19M chr1 118912052 118912071 + 19M snRNA U5 19686 100087314


In [319]:
cat npcrna.csv | awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13,$14}' OFS='\t' > npcrna1.tsv

In [320]:
head npcrna1.tsv

chr6	71971138	71971157	+	19M	chr13	124590001	124590020	-	19M	snRNA	U5	19686	100002016
chr14	40286509	40286528	-	19M	chr6	100028477	100028496	+	19M	snRNA	U4	8808	100004528
chr1	236380016	236380035	-	19M	chr18	11678219	11678239	+	20M	ribozyme	RNase_MRP	1300	100031442
chr14	40286508	40286527	-	19M	chr11	13656391	13656410	-	19M	snRNA	U4	8808	100036647
chr1	163335027	163335046	+	19M	chr14	48515712	48515731	-	19M	snRNA	U5	19686	100043587
chr14	40286550	40286569	-	19M	chr13	199193288	199193307	-	19M	snRNA	U4	8808	100059528
chr14	40286550	40286569	-	19M	chr6	54532506	54532526	+	20M	snRNA	U4	8808	100070833
chr14	40286550	40286569	-	19M	chr6	54532506	54532526	+	20M	snRNA	U4	8808	100070833
chr6	54567220	54567239	+	19M	chr18	16638200	16638219	+	19M	snoRNA	SNORD33	1228	100084671
chr6	71971138	71971157	+	19M	chr1	118912052	118912071	+	19M	snRNA	U5	19686	100087314


In [323]:
cat fpcrna_contacts1.csv npcrna1.tsv > filtered_mrna_contacts.tsv